# Importing librairies

In [1]:
import pandas as pd
import numpy as np

Path to data

In [2]:
DATA_PATH = "../data/"

In [3]:
df = pd.read_csv(DATA_PATH+"data_comment_treated.csv").drop(['Unnamed: 0'],axis=1)
df.head(20)

,cell_type,source,title
0,markdown,House Prices: Advanced Regression Techniques P...,1-house-prices-solution-top-1.ipynb
1,markdown,Load packages,1-house-prices-solution-top-1.ipynb
2,markdown,This Python 3 environment comes with many help...,1-house-prices-solution-top-1.ipynb
3,markdown,It is defined by the kaggle python docker image:,1-house-prices-solution-top-1.ipynb
4,markdown,"For example, here s several helpful packages t...",1-house-prices-solution-top-1.ipynb
5,markdown,linear algebra,1-house-prices-solution-top-1.ipynb
6,code,import numpy as np,1-house-prices-solution-top-1.ipynb
7,markdown,"data processing, CSV file I O e.g. pd.read csv",1-house-prices-solution-top-1.ipynb
8,code,import pandas as pd,1-house-prices-solution-top-1.ipynb
9,markdown,Input data files are available in the .. input...,1-house-prices-solution-top-1.ipynb


# Handling missing values

In [4]:
df.isna().sum()

cell_type      0
source       238
title          0
dtype: int64

In [5]:
df =df.dropna(subset=['source'])

In [6]:
df.isna().sum()

cell_type    0
source       0
title        0
dtype: int64

In [7]:
df = df.reset_index(drop=True)

# First strategy 

In [9]:
def reorganise_cells(df,i,new_df,limit,ntb):

    if df.loc[i]['cell_type'] == 'markdown' and i < limit:
        grp_md(i,df,new_df,limit,ntb)

    elif df.loc[i]['cell_type'] == 'code' and i < limit:
        grp_cd(i,df,new_df,limit,ntb)       
   

In [10]:
def grp_md(i,df,new_df,limit,ntb):
    
    cell_md = ""
    
    while i < limit and df.loc[i]['cell_type'] == 'markdown':
        cell_md += str(df.loc[i]['source']) +" "
        i += 1

    new_df.loc[len(new_df)] = ['markdown',cell_md,ntb]


    if i < limit :
        reorganise_cells(df,i,new_df,limit,ntb)

In [11]:
def grp_cd(i,df,new_df,limit,ntb):
    cell_cd = ""

    
    while i < limit and df.loc[i]['cell_type'] == 'code':
        cell_cd += str(df.loc[i]['source']) +" "
        i += 1

    new_df.loc[len(new_df)] = ['code',cell_cd,ntb]



    if i < limit :
        reorganise_cells(df,i,new_df,limit,ntb)

In [12]:
interm_df = pd.DataFrame(columns=['cell_type','source','title'])

In [8]:
List_ntb = []
for i in df.index:
    List_ntb.append(df.loc[i]['title'])

List_ntb_only = np.unique(List_ntb)


In [14]:
for ntb in List_ntb_only:
    reorganise_cells(df[df['title'] == ntb].reset_index(drop=True),0,interm_df,len(df[df['title'] == ntb]),ntb)


In [15]:
interm_df

,cell_type,source,title
0,markdown,House Prices: Advanced Regression Techniques P...,1-house-prices-solution-top-1.ipynb
1,code,import numpy as np,1-house-prices-solution-top-1.ipynb
2,markdown,"data processing, CSV file I O e.g. pd.read csv",1-house-prices-solution-top-1.ipynb
3,code,import pandas as pd,1-house-prices-solution-top-1.ipynb
4,markdown,Input data files are available in the .. input...,1-house-prices-solution-top-1.ipynb
...,...,...,...
15318,markdown,Visualizing scikit model performance,your-first-nlp-competition-submission.ipynb
15319,code,training_accuracy = accuracy_score(train_label...,your-first-nlp-competition-submission.ipynb
15320,markdown,from sklearn.neighbors import KNeighborsClassi...,your-first-nlp-competition-submission.ipynb
15321,code,test_vectorizer =tfidf.transform( data_clean_t...,your-first-nlp-competition-submission.ipynb


# Constructing pairs

We can distingue four types of ntb :
- Starting with md ending with cd .
- Starting with md ending with md .
- Starting with cd ending with cd .
- Starting with cd ending with md .

In [16]:
l1 = []
l2 = []
l3 = []
l4 = []

for ntb in List_ntb_only:
    data = interm_df[interm_df['title'] == ntb].reset_index(drop=True)
    if data.loc[0]['cell_type']=="markdown" and data.loc[len(data)-1]['cell_type']=="code":
        l1.append(ntb)
    elif data.loc[0]['cell_type']=="markdown" and data.loc[len(data)-1]['cell_type']=="markdown":
        l2.append(ntb)
    elif data.loc[0]['cell_type']=="code" and data.loc[len(data)-1]['cell_type']=="markdown":
        l3.append(ntb)
    elif data.loc[0]['cell_type']=="code" and data.loc[len(data)-1]['cell_type']=="code":
        l4.append(ntb)

print(len(l1),len(l2),len(l3),len(l4))

106 150 6 17


In [17]:
new_df = pd.DataFrame(columns=['markdown','code','title'])

In [18]:
def construct_pairs(data,limit,new_df,l1,l2,l3,l4,ntb):
    if ntb in l1 :
        grp_pairs_md_cd(data,ntb,limit,new_df,0)
    
    elif ntb in l2 :
        grp_pairs_md_cd(data[:len(data)-1],ntb,limit-1,new_df,0)

    elif ntb in l4 :
        grp_pairs_md_cd(data[1:].reset_index(drop=True),ntb,limit-1,new_df,0)
    
    elif ntb in l3 :
        grp_pairs_md_cd(data[1:len(data)-1].reset_index(drop=True),ntb,limit-2,new_df,0)


In [19]:
def grp_pairs_md_cd(data,title,limit,new_df,counter):
    i = counter
    while i < limit-1 and data.loc[i]['cell_type'] == 'markdown' and data.loc[i+1]['cell_type'] == 'code':
        new_df.loc[len(new_df)] = [data.loc[i]['source'],data.loc[i+1]['source'],title]
        i+=2

    if i < limit :
         grp_pairs_md_cd(data[i:],title,limit,new_df,i)


In [20]:
for ntb in List_ntb_only :
    data = interm_df[interm_df['title'] == ntb].reset_index(drop=True)
    construct_pairs(data,len(data),new_df,l1,l2,l3,l4,ntb)

In [21]:
new_df

,markdown,code,title
0,House Prices: Advanced Regression Techniques P...,import numpy as np,1-house-prices-solution-top-1.ipynb
1,"data processing, CSV file I O e.g. pd.read csv",import pandas as pd,1-house-prices-solution-top-1.ipynb
2,Input data files are available in the .. input...,from datetime import datetime,1-house-prices-solution-top-1.ipynb
3,for some statistics,from scipy . stats import skew from scipy ....,1-house-prices-solution-top-1.ipynb
4,Any results you write to the current directory...,train = pd.read_csv('../input/house-prices-adv...,1-house-prices-solution-top-1.ipynb
...,...,...,...
7567,"If the word exists in the vocabulary, this com...",test_features = tfidf.transform(X_test).toarra...,your-first-nlp-competition-submission.ipynb
7568,reduce the matrix dimensionality in order to ...,"""""""from sklearn import feature_selection \r\ny...",your-first-nlp-competition-submission.ipynb
7569,Model training Model : Multinomial NB,import pandas as pd\r\nfrom sklearn.naive_baye...,your-first-nlp-competition-submission.ipynb
7570,Visualizing scikit model performance,training_accuracy = accuracy_score(train_label...,your-first-nlp-competition-submission.ipynb


In [22]:
new_df.to_csv(DATA_PATH+"data_pairs_1.csv")

# Second strategy

In [13]:
df

,cell_type,source,title
0,markdown,House Prices: Advanced Regression Techniques P...,1-house-prices-solution-top-1.ipynb
1,markdown,Load packages,1-house-prices-solution-top-1.ipynb
2,markdown,This Python 3 environment comes with many help...,1-house-prices-solution-top-1.ipynb
3,markdown,It is defined by the kaggle python docker image:,1-house-prices-solution-top-1.ipynb
4,markdown,"For example, here s several helpful packages t...",1-house-prices-solution-top-1.ipynb
...,...,...,...
35882,code,submission_df,your-first-nlp-competition-submission.ipynb
35883,code,submission_df['target'].value_counts(),your-first-nlp-competition-submission.ipynb
35884,code,submission = submission_df.to_csv('Result.csv'...,your-first-nlp-competition-submission.ipynb
35885,markdown,"if you liked this kernel,please upvote it 3",your-first-nlp-competition-submission.ipynb


In [20]:
new_df_2 = pd.DataFrame(columns = ['markdown','code','title'])

In [21]:
def grp_pairs(data,title,limit,new_df):
    i = 0
    
    if i < limit-1 and data.loc[i]['cell_type'] == 'markdown' and data.loc[i+1]['cell_type'] == 'code':
        new_df.loc[len(new_df)] = [data.loc[i]['source'],data.loc[i+1]['source'],title]
        i+=2
        if i < limit :
            grp_pairs(data[i:].reset_index(drop=True),title,len(data[i:]),new_df)

    else :
            i+=1
            if i < limit :
                grp_pairs(data[i:].reset_index(drop=True),title,len(data[i:]),new_df)


In [22]:
for ntb in List_ntb_only :
    data = df[df['title'] == ntb].reset_index(drop=True)

    grp_pairs(data,ntb,len(df),new_df_2)

In [23]:
new_df_2

,markdown,code,title
0,linear algebra,import numpy as np,1-house-prices-solution-top-1.ipynb
1,"data processing, CSV file I O e.g. pd.read csv",import pandas as pd,1-house-prices-solution-top-1.ipynb
2,"For example, running this by clicking run or p...",from datetime import datetime,1-house-prices-solution-top-1.ipynb
3,for some statistics,from scipy . stats import skew,1-house-prices-solution-top-1.ipynb
4,Load data,train = pd.read_csv('../input/house-prices-adv...,1-house-prices-solution-top-1.ipynb
...,...,...,...
7567,meaning that the Nth feature of the matrix is ...,test_features = tfidf.transform(X_test).toarra...,your-first-nlp-competition-submission.ipynb
7568,in order to reduce the dimensionality of our m...,"""""""from sklearn import feature_selection \r\ny...",your-first-nlp-competition-submission.ipynb
7569,Model : Multinomial NB,import pandas as pd\r\nfrom sklearn.naive_baye...,your-first-nlp-competition-submission.ipynb
7570,Visualizing scikit model performance,training_accuracy = accuracy_score(train_label...,your-first-nlp-competition-submission.ipynb


In [24]:
new_df_2.to_csv(DATA_PATH+"data_pairs_2.csv")